In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
data = pd.read_csv('/content/CA_Industrial_RE_sales.csv')

In [ ]:
data.head()

,Property Type,Building SF,Star Rating,Sale Price,Price Per SF,Sale Date,Percent Leased,Secondary Type,Building Class,Year Built,...,Market Cap Rate,Under Construction SF,Net Absorption SF,Population,Industrial Employment,Sales Volume Transactions Growth,Net Absorption SF Growth,Under Construction SF Growth,Population Growth,Industrial Employment Growth
0,Industrial,229608,5 Star,61035224.0,265.8235,2020-01-24,1.0,Food Processing,A,2015.0,...,0.050,6479589.0,-1246637.0,10046150.0,495296.0,0.001448,-0.025522,0.000803,-1.416445e-05,-0.000017
1,Industrial,39300,3 Star,6500000.0,165.3944,2020-01-24,1.0,Manufacturing,C,1980.0,...,0.050,23090656.0,3544932.0,4586135.0,212755.0,0.003111,-0.004820,-0.001148,1.679172e-05,0.000051
2,Industrial,107359,3 Star,6240000.0,58.1227,2020-01-24,0.9,Truck Terminal,C,1960.0,...,0.050,6479589.0,-1246637.0,10046150.0,495296.0,0.001448,-0.025522,0.000803,-1.416445e-05,-0.000017
3,Industrial,6370,2 Star,2700000.0,423.8618,2020-01-24,1.0,Warehouse,C,1980.0,...,0.050,6479589.0,-1246637.0,10046150.0,495296.0,0.001448,-0.025522,0.000803,-1.416445e-05,-0.000017
4,Industrial,4670,3 Star,1580000.0,338.3297,2020-01-24,1.0,Warehouse,C,2004.0,...,0.051,901929.0,302419.0,3188158.0,265966.0,0.000900,0.022360,-0.000842,-3.136598e-08,-0.000054


In [ ]:
data.columns

Index(['Property Type', 'Building SF', 'Star Rating', 'Sale Price',
       'Price Per SF', 'Sale Date', 'Percent Leased', 'Secondary Type',
       'Building Class', 'Year Built', 'Land Area SF', 'Drive Ins', 'Zoning',
       'Market', 'Submarket Name', 'Age', 'Building Materials',
       'Building Tax Expenses', 'Coverage', 'Improvement Ratio', 'Latitude',
       'Longitude', 'Number Of Parking Spaces', 'Parking Ratio',
       'Distance_to_Nearest_Port', 'distance_public_transport',
       'distance_to_highway', 'distance_to_nearest_airport',
       'distance_to_nearest_rail', 'distance_to_nearest_dc',
       'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate',
       'Market Sale Price Growth', 'Sales Volume Transactions',
       'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF',
       'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth'

In [ ]:
data = data.dropna(subset=['Sale Date'])

In [ ]:
def add_macro_economic_data(df):
    # Create a copy of the dataframe
    df = df.copy()

    # Generate quarterly dates
    dates = pd.date_range(start='2019-01-01', end='2024-09-30', freq='QE')
    gdp_values = [
        21694.3, 21812.3, 21961.1, 22024.5,  # 2019 Q1-Q4
        21481.3, 19520.1, 21170.3, 21477.6,  # 2020 Q1-Q4
        22038.2, 22741.0, 23202.3, 24008.5,  # 2021 Q1-Q4
        24382.2, 24740.0, 25248.5, 25723.9,  # 2022 Q1-Q4
        26378.5, 26814.7, 27165.5, 27630.3,  # 2023 Q1-Q4
        27892.0, 28230.0, 28550.0            # 2024 Q1-Q3
    ]

    gdp_df = pd.DataFrame({
        'Date': dates,
        'GDP': gdp_values
    })

    # Create inflation data
    inflation_dates = pd.date_range(start='2019-01-01', end='2024-10-31', freq='ME')
    inflation_rates = [
        1.6, 1.5, 1.9, 2.0, 1.8, 1.6, 1.8, 1.7, 1.7, 1.8, 2.1, 2.3,  # 2019
        2.5, 2.3, 1.5, 0.3, 0.1, 0.6, 1.0, 1.3, 1.4, 1.2, 1.2, 1.4,  # 2020
        1.4, 1.7, 2.6, 4.2, 5.0, 5.4, 5.4, 5.3, 5.4, 6.2, 6.8, 7.0,  # 2021
        7.5, 7.9, 8.5, 8.3, 8.6, 9.1, 8.5, 8.3, 8.2, 7.7, 7.1, 6.5,  # 2022
        6.4, 6.0, 5.0, 4.9, 4.0, 3.0, 3.2, 3.7, 3.7, 3.2, 3.1, 3.4,  # 2023
        3.1, 3.2, 3.5, 3.4, 3.3, 3.0, 2.9, 2.5, 2.4, 2.6             # 2024 Jan-Oct
    ]

    inflation_df = pd.DataFrame({
        'Date': inflation_dates,
        'Inflation_Rate': inflation_rates
    })

    # Convert Sale Date to datetime
    df['Sale Date'] = pd.to_datetime(df['Sale Date'])

    # Get quarter start for each sale date
    df['Quarter_Start'] = df['Sale Date'].dt.to_period('Q').dt.start_time

    # Merge GDP data
    df = pd.merge_asof(
        df.sort_values('Sale Date'),
        gdp_df.rename(columns={'Date': 'Quarter_Start'}),
        on='Quarter_Start',
        direction='backward'
    )

    # Merge inflation data
    df = pd.merge_asof(
        df.sort_values('Sale Date'),
        inflation_df.rename(columns={'Date': 'Sale Date'}),
        on='Sale Date',
        direction='backward'
    )

    # Drop intermediate column
    df = df.drop('Quarter_Start', axis=1)

    return df

# Apply the function to your existing dataframe
enhanced_data = add_macro_economic_data(data)

# Display some info about the result
print("\nShape of enhanced data:", enhanced_data.shape)
print("\nColumns in enhanced data:", enhanced_data.columns.tolist())
print("\nSample of enhanced data:")
print(enhanced_data[['Sale Date', 'GDP', 'Inflation_Rate']].head())


Shape of enhanced data: (9292, 47)

Columns in enhanced data: ['Property Type', 'Building SF', 'Star Rating', 'Sale Price', 'Price Per SF', 'Sale Date', 'Percent Leased', 'Secondary Type', 'Building Class', 'Year Built', 'Land Area SF', 'Drive Ins', 'Zoning', 'Market', 'Submarket Name', 'Age', 'Building Materials', 'Building Tax Expenses', 'Coverage', 'Improvement Ratio', 'Latitude', 'Longitude', 'Number Of Parking Spaces', 'Parking Ratio', 'Distance_to_Nearest_Port', 'distance_public_transport', 'distance_to_highway', 'distance_to_nearest_airport', 'distance_to_nearest_rail', 'distance_to_nearest_dc', 'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate', 'Market Sale Price Growth', 'Sales Volume Transactions', 'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF', 'Population', 'Industrial Employment', 'Sales Volume Transactions Growth', 'Net Absorption SF Growth', 'Under Construction SF Growth', 'Population Growth', 'Industrial Employment Growth', 'GDP', 

In [ ]:
enhanced_data.head(100)

,Property Type,Building SF,Star Rating,Sale Price,Price Per SF,Sale Date,Percent Leased,Secondary Type,Building Class,Year Built,...,Net Absorption SF,Population,Industrial Employment,Sales Volume Transactions Growth,Net Absorption SF Growth,Under Construction SF Growth,Population Growth,Industrial Employment Growth,GDP,Inflation_Rate
0,Industrial,5086,1 Star,172019.0,33.8220,2019-12-04,1.00,Warehouse,C,NaN,...,218422.0,1005563.0,45668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
1,Industrial,8782,2 Star,297026.0,33.8221,2019-12-04,1.00,Warehouse,C,NaN,...,218422.0,1005563.0,45668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
2,Industrial,2475,2 Star,1120000.0,452.5252,2019-12-04,NaN,Warehouse,B,1982.0,...,118193.0,1651662.0,86281.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
3,Industrial,5500,1 Star,2220000.0,403.6363,2019-12-04,0.00,Warehouse,C,1964.0,...,803162.0,10060400.0,496116.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
4,Industrial,16950,2 Star,3200000.0,188.7905,2019-12-04,1.00,Warehouse,B,1988.0,...,6844008.0,4578447.0,211668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Industrial,4588,2 Star,750000.0,163.4699,2019-12-20,0.00,Distribution,C,1960.0,...,803162.0,10060400.0,496116.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
96,Industrial,14348,3 Star,3090000.0,215.3610,2019-12-20,1.00,Warehouse,B,2001.0,...,93456.0,3188168.0,267404.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
97,Industrial,5000,2 Star,300000.0,60.0000,2019-12-20,1.00,Warehouse,C,1947.0,...,-29524.0,906613.0,38987.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1
98,Industrial,5000,2 Star,300000.0,60.0000,2019-12-20,0.05,Warehouse,C,1947.0,...,-29524.0,906613.0,38987.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1


In [ ]:
data.columns

Index(['Property Type', 'Building SF', 'Star Rating', 'Sale Price',
       'Price Per SF', 'Sale Date', 'Percent Leased', 'Secondary Type',
       'Building Class', 'Year Built', 'Land Area SF', 'Drive Ins', 'Zoning',
       'Market', 'Submarket Name', 'Age', 'Building Materials',
       'Building Tax Expenses', 'Coverage', 'Improvement Ratio', 'Latitude',
       'Longitude', 'Number Of Parking Spaces', 'Parking Ratio',
       'Distance_to_Nearest_Port', 'distance_public_transport',
       'distance_to_highway', 'distance_to_nearest_airport',
       'distance_to_nearest_rail', 'distance_to_nearest_dc',
       'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate',
       'Market Sale Price Growth', 'Sales Volume Transactions',
       'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF',
       'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth'

In [ ]:
enhanced_data.isna().sum()

,0
Property Type,0
Building SF,0
Star Rating,2
Sale Price,0
Price Per SF,0
Sale Date,0
Percent Leased,599
Secondary Type,274
Building Class,35
Year Built,412


In [ ]:
data.isna().sum()

,0
Property Type,0
Building SF,0
Star Rating,2
Sale Price,0
Price Per SF,0
Sale Date,0
Percent Leased,599
Secondary Type,274
Building Class,35
Year Built,412


In [ ]:
# Write the enhanced dataset to a CSV file
enhanced_data.to_csv('real_estate_data.csv', index=False)
print("Data has been saved to 'enhanced_real_estate_data.csv'")

Data has been saved to 'enhanced_real_estate_data.csv'


In [ ]:
enhanced_data.shape

(9292, 47)

In [ ]:
import pandas_datareader as pdr
from datetime import datetime, timedelta

# Get Treasury data from FRED
treasury_data = pdr.get_data_fred('DGS10', start='2019-01-01')  # Adjust start date based on your earliest sale date
treasury_data.columns = ['Treasury_10Y']

# Convert Sale Date to datetime if it's not already
enhanced_data['Sale Date'] = pd.to_datetime(enhanced_data['Sale Date'])

# Merge the treasury data with your main dataset
# Since treasury data doesn't include weekends/holidays, we'll forward fill the values
treasury_data = treasury_data.fillna(method='ffill')
enhanced_data['Treasury_10Y'] = enhanced_data['Sale Date'].map(treasury_data['Treasury_10Y'])

# If any dates don't have a match, use the closest previous date
enhanced_data['Treasury_10Y'] = enhanced_data['Treasury_10Y'].fillna(method='ffill')

<ipython-input-13-d7af86262a33>:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  treasury_data = treasury_data.fillna(method='ffill')
<ipython-input-13-d7af86262a33>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  enhanced_data['Treasury_10Y'] = enhanced_data['Treasury_10Y'].fillna(method='ffill')


In [ ]:
enhanced_data.columns

Index(['Property Type', 'Building SF', 'Star Rating', 'Sale Price',
       'Price Per SF', 'Sale Date', 'Percent Leased', 'Secondary Type',
       'Building Class', 'Year Built', 'Land Area SF', 'Drive Ins', 'Zoning',
       'Market', 'Submarket Name', 'Age', 'Building Materials',
       'Building Tax Expenses', 'Coverage', 'Improvement Ratio', 'Latitude',
       'Longitude', 'Number Of Parking Spaces', 'Parking Ratio',
       'Distance_to_Nearest_Port', 'distance_public_transport',
       'distance_to_highway', 'distance_to_nearest_airport',
       'distance_to_nearest_rail', 'distance_to_nearest_dc',
       'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate',
       'Market Sale Price Growth', 'Sales Volume Transactions',
       'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF',
       'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth'

In [ ]:
enhanced_data.head(100)

,Property Type,Building SF,Star Rating,Sale Price,Price Per SF,Sale Date,Percent Leased,Secondary Type,Building Class,Year Built,...,Population,Industrial Employment,Sales Volume Transactions Growth,Net Absorption SF Growth,Under Construction SF Growth,Population Growth,Industrial Employment Growth,GDP,Inflation_Rate,Treasury_10Y
0,Industrial,5086,1 Star,172019.0,33.8220,2019-12-04,1.00,Warehouse,C,NaN,...,1005563.0,45668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.77
1,Industrial,8782,2 Star,297026.0,33.8221,2019-12-04,1.00,Warehouse,C,NaN,...,1005563.0,45668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.77
2,Industrial,2475,2 Star,1120000.0,452.5252,2019-12-04,NaN,Warehouse,B,1982.0,...,1651662.0,86281.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.77
3,Industrial,5500,1 Star,2220000.0,403.6363,2019-12-04,0.00,Warehouse,C,1964.0,...,10060400.0,496116.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.77
4,Industrial,16950,2 Star,3200000.0,188.7905,2019-12-04,1.00,Warehouse,B,1988.0,...,4578447.0,211668.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Industrial,4588,2 Star,750000.0,163.4699,2019-12-20,0.00,Distribution,C,1960.0,...,10060400.0,496116.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.92
96,Industrial,14348,3 Star,3090000.0,215.3610,2019-12-20,1.00,Warehouse,B,2001.0,...,3188168.0,267404.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.92
97,Industrial,5000,2 Star,300000.0,60.0000,2019-12-20,1.00,Warehouse,C,1947.0,...,906613.0,38987.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.92
98,Industrial,5000,2 Star,300000.0,60.0000,2019-12-20,0.05,Warehouse,C,1947.0,...,906613.0,38987.0,NaN,NaN,NaN,NaN,NaN,21961.1,2.1,1.92


In [ ]:
enhanced_data.columns

Index(['Property Type', 'Building SF', 'Star Rating', 'Sale Price',
       'Price Per SF', 'Sale Date', 'Percent Leased', 'Secondary Type',
       'Building Class', 'Year Built', 'Land Area SF', 'Drive Ins', 'Zoning',
       'Market', 'Submarket Name', 'Age', 'Building Materials',
       'Building Tax Expenses', 'Coverage', 'Improvement Ratio', 'Latitude',
       'Longitude', 'Number Of Parking Spaces', 'Parking Ratio',
       'Distance_to_Nearest_Port', 'distance_public_transport',
       'distance_to_highway', 'distance_to_nearest_airport',
       'distance_to_nearest_rail', 'distance_to_nearest_dc',
       'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate',
       'Market Sale Price Growth', 'Sales Volume Transactions',
       'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF',
       'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth'

In [ ]:
columns_to_keep = ['Building SF', 'Price Per SF', 'Sale Date', 'Secondary Type', 'Building Class',
                   'Age', 'Building Tax Expenses','Coverage','Improvement Ratio',
                  'Distance_to_Nearest_Port', 'distance_public_transport',
       'distance_to_highway', 'distance_to_nearest_airport',
       'distance_to_nearest_rail', 'distance_to_nearest_dc',
       'distance_to_residential', 'Market Asking Rent Growth', 'Vacancy Rate',
       'Market Sale Price Growth', 'Sales Volume Transactions',
       'Market Cap Rate', 'Under Construction SF', 'Net Absorption SF',
       'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth',
       'Industrial Employment Growth', 'GDP', 'Inflation_Rate',
       'Treasury_10Y', 'Sale Price'

                   ]

In [ ]:
model_data = enhanced_data[columns_to_keep]

In [ ]:
model_data.columns

Index(['Building SF', 'Price Per SF', 'Sale Date', 'Secondary Type',
       'Building Class', 'Age', 'Building Tax Expenses', 'Coverage',
       'Improvement Ratio', 'Distance_to_Nearest_Port',
       'distance_public_transport', 'distance_to_highway',
       'distance_to_nearest_airport', 'distance_to_nearest_rail',
       'distance_to_nearest_dc', 'distance_to_residential',
       'Market Asking Rent Growth', 'Vacancy Rate', 'Market Sale Price Growth',
       'Sales Volume Transactions', 'Market Cap Rate', 'Under Construction SF',
       'Net Absorption SF', 'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth',
       'Industrial Employment Growth', 'GDP', 'Inflation_Rate', 'Treasury_10Y',
       'Sale Price'],
      dtype='object')

In [ ]:
model_data.isna().sum()

,0
Building SF,0
Price Per SF,0
Sale Date,0
Secondary Type,274
Building Class,35
Age,412
Building Tax Expenses,305
Coverage,63
Improvement Ratio,413
Distance_to_Nearest_Port,0


In [ ]:
model_data = model_data.dropna()

In [ ]:
model_data.isna().sum()

,0
Building SF,0
Price Per SF,0
Sale Date,0
Secondary Type,0
Building Class,0
Age,0
Building Tax Expenses,0
Coverage,0
Improvement Ratio,0
Distance_to_Nearest_Port,0


In [ ]:
model_data.shape

(7359, 34)

In [ ]:
model_data.to_csv('model_industrial_data2.csv', index=False)

In [ ]:
model_data.columns

Index(['Building SF', 'Price Per SF', 'Sale Date', 'Secondary Type',
       'Building Class', 'Age', 'Building Tax Expenses', 'Coverage',
       'Improvement Ratio', 'Distance_to_Nearest_Port',
       'distance_public_transport', 'distance_to_highway',
       'distance_to_nearest_airport', 'distance_to_nearest_rail',
       'distance_to_nearest_dc', 'distance_to_residential',
       'Market Asking Rent Growth', 'Vacancy Rate', 'Market Sale Price Growth',
       'Sales Volume Transactions', 'Market Cap Rate', 'Under Construction SF',
       'Net Absorption SF', 'Population', 'Industrial Employment',
       'Sales Volume Transactions Growth', 'Net Absorption SF Growth',
       'Under Construction SF Growth', 'Population Growth',
       'Industrial Employment Growth', 'GDP', 'Inflation_Rate', 'Treasury_10Y',
       'Sale Price'],
      dtype='object')